In [ ]:
import sqlite3
import csv

#create db
conn = sqlite3.connect('XYZ_DATABASE.db')
cursor = conn.cursor()

#create business tables

cursor.execute("CREATE TABLE Sales(sales_id, customer_id)")
cursor.execute("CREATE TABLE Customer(customer_id, age)")
cursor.execute("CREATE TABLE Orders(order_id, sales_id, item_id, quantity)")
cursor.execute("CREATE TABLE Items(item_id, item_name)")


# Execute SQL query to retrieve the required data
cursor.execute('''
    SELECT C.CustomerID, C.Age, I.Item, SUM(IFNULL(O.Quantity, 0)) AS TotalQuantity
    FROM Customer C
    JOIN Orders O ON C.CustomerID = O.CustomerID
    JOIN Items I ON O.ItemID = I.ItemID
    WHERE C.Age BETWEEN 18 AND 35
    GROUP BY C.CustomerID, C.Age, I.Item
    HAVING TotalQuantity > 0
''')

# Fetch all the results
rows = cursor.fetchall()

# Write results to a CSV file
with open('output.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=';')    r
    csv_writer.writerow(['Customer', 'Age', 'Item', 'Quantity'])    
    csv_writer.writerows(rows)


conn.commit()
conn.close()


In [ ]:
import sqlite3
import pandas as pd

# Read the data from SQLite database into a DataFrame
conn = sqlite3.connect('XYZ_DATABASE.db')
query = '''
    SELECT C.CustomerID, C.Age, I.Item, IFNULL(O.Quantity, 0) AS Quantity
    FROM Customer C
    JOIN Orders O ON C.CustomerID = O.CustomerID
    JOIN Items I ON O.ItemID = I.ItemID
    WHERE C.Age BETWEEN 18 AND 35
'''
df = pd.read_sql_query(query, conn)

result_df = df.groupby(['CustomerID', 'Age', 'Item'])['Quantity'].sum().reset_index()


result_df = result_df[result_df['Quantity'] > 0]

result_df.to_csv('output.csv', sep=';', index=False)


conn.close()
